In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [2]:
df = pd.read_csv("diagnosis-of-covid-19-and-its-clinical-spectrum.csv")
df = df.dropna(thresh=56)
df = df.dropna(axis=1)
df

,patient_id,patient_age_quantile,sars_cov_2_exam_result,patient_addmited_to_regular_ward_1_yes_0_no,patient_addmited_to_semi_intensive_unit_1_yes_0_no,patient_addmited_to_intensive_care_unit_1_yes_0_no,hematocrit,hemoglobin,platelets,mean_platelet_volume,red_blood_cells,lymphocytes,mean_corpuscular_hemoglobin_concentration_mchc,leukocytes,basophils,mean_corpuscular_hemoglobin_mch,eosinophils,mean_corpuscular_volume_mcv,red_blood_cell_distribution_width_rdw
8,8bb9d64f0215244,1,negative,f,t,f,-1.571682,-0.774212,1.429667,-1.672222,-0.850035,-0.005738,3.331071,0.364550,-0.223767,0.178175,1.018625,-1.336024,-0.978899
50,11e4956b770f080,17,negative,f,f,f,1.014726,0.666876,-0.203368,0.459449,0.295938,-0.261609,-0.950790,-0.083479,-0.223767,0.544077,0.133698,1.167670,-0.359703
55,ac357896cbf35fc,19,negative,f,t,f,0.625621,0.353596,-0.127997,0.796029,0.190156,-1.592140,-0.851210,0.556563,0.081693,0.230447,-0.624811,0.686961,-0.359703
62,2e90e489408ca11,11,negative,f,f,f,0.740064,0.541564,-0.065188,0.683835,-0.215342,0.966573,-0.552476,0.350636,0.081693,1.328151,0.555092,1.808615,0.790231
87,26000ea3dba0de1,17,negative,f,f,f,0.442512,0.228284,-0.680717,0.459449,0.736697,-1.165688,-0.652057,-0.589947,-0.529226,-0.919529,-0.624811,-0.715108,0.082579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5583,73811966f4b1141,19,negative,f,f,f,-1.731902,-1.588740,-0.856582,-0.101517,-1.555249,3.218241,0.144572,-1.928470,1.608988,0.073631,0.344395,-0.034103,-0.359703
5584,5b57f3b7c8e67d0,0,negative,f,f,f,-1.869234,-1.776707,0.022745,-0.550290,-0.797144,2.570034,-0.054585,1.569499,-1.140144,-1.860418,-0.835508,-2.057088,0.790231
5602,c5b44ff9c7782fd,19,negative,f,f,f,0.190738,0.165628,-0.102873,0.908221,0.384090,-1.583611,-0.054585,-0.328365,-0.834685,-0.449085,0.892207,-0.474754,0.347948
5615,2733fac0d3f7138,15,negative,f,f,f,0.717175,1.105468,-0.492289,-0.213711,0.613284,0.002791,1.538664,-0.550988,-0.223767,0.700891,-0.709090,0.005955,-1.155812


In [3]:
df_positive = df[df.sars_cov_2_exam_result=='positive']
df_negative = df[df.sars_cov_2_exam_result=='negative']

df_positive_upsampled = resample(df_positive, 
                                 replace=True,
                                 n_samples=int((len(df_positive)+len(df_negative))/2),#len(df_positive),
                                 random_state=5)

df_negative_upsampled = resample(df_negative, 
                                 replace=True,
                                 n_samples=len(df_positive_upsampled),
                                 random_state=5)
 
df_upsampled = pd.concat([df_positive_upsampled, df_negative_upsampled])
 
df_upsampled.sars_cov_2_exam_result.value_counts()

negative    62
positive    62
Name: sars_cov_2_exam_result, dtype: int64

In [4]:
df_train, df_test = train_test_split(df_upsampled,train_size=0.8,random_state=5)
y_train, y_test = df_train.sars_cov_2_exam_result, df_test.sars_cov_2_exam_result
x_train, x_test = df_train.drop(['patient_age_quantile','patient_id','sars_cov_2_exam_result','patient_addmited_to_intensive_care_unit_1_yes_0_no','patient_addmited_to_regular_ward_1_yes_0_no','patient_addmited_to_semi_intensive_unit_1_yes_0_no'], axis=1), df_test.drop(['patient_age_quantile','patient_id','sars_cov_2_exam_result','patient_addmited_to_intensive_care_unit_1_yes_0_no','patient_addmited_to_regular_ward_1_yes_0_no','patient_addmited_to_semi_intensive_unit_1_yes_0_no'], axis=1)

clf = RandomForestClassifier()
clf.fit(x_train, y_train)

pred_y = clf.predict(x_train)

print( np.unique( pred_y ) )

print( accuracy_score(y_train, pred_y) )

prob_y = clf.predict_proba(x_train)
prob_y = [p[1] for p in prob_y]
print( roc_auc_score(y_train, prob_y) )

['negative' 'positive']
1.0
1.0


In [5]:
pred_y = clf.predict(x_test)

print( np.unique( pred_y ) )

print( accuracy_score(y_test, pred_y) )

prob_y = clf.predict_proba(x_test)
prob_y = [p[1] for p in prob_y]
print( roc_auc_score(y_test, prob_y) )

['negative' 'positive']
0.88
0.935064935064935


In [6]:
categorize_results = {"positive": 1, "negative": 0}

In [7]:
y_train = y_train.replace(categorize_results)
y_test = y_test.replace(categorize_results)

In [8]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [92]:
#'linear','elu','selu','relu','sigmoid','hard_sigmoid','tanh','softmax','softplus','softsign','exponential'
#'mean_squared_error','mean_absolute_error','mean_absolute_percentage_error','mean_squared_logarithmic_error','squared_hinge','hinge','categorical_hinge','logcosh','huber_loss','binary_crossentropy','kullback_leibler_divergence','poisson','cosine_proximity'
#'sgd','rmsprop','adagrad','adadelta','adam','adamT','adamax','nadam'
model = Sequential()
model.add(Dense(16,input_dim=(13),activation='tanh'))
model.add(Dense(8,activation='relu'))
model.add(Dense(4,activation='exponential'))
model.add(Dense(4,activation='softsign'))
model.add(Dense(2,activation='linear'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='mse',optimizer='nadam',metrics=['binary_accuracy'])

In [93]:
model.fit(x_train,y_train,epochs=1000)

Epoch 1/1000
99/99 [==============================] - 0s 2ms/step - loss: 0.2508 - binary_accuracy: 0.4848
Epoch 2/1000
99/99 [==============================] - 0s 81us/step - loss: 0.2497 - binary_accuracy: 0.5354
Epoch 3/1000
99/99 [==============================] - 0s 91us/step - loss: 0.2489 - binary_accuracy: 0.6061
Epoch 4/1000
99/99 [==============================] - 0s 91us/step - loss: 0.2485 - binary_accuracy: 0.5556
Epoch 5/1000
99/99 [==============================] - 0s 91us/step - loss: 0.2482 - binary_accuracy: 0.5051
Epoch 6/1000
99/99 [==============================] - 0s 91us/step - loss: 0.2486 - binary_accuracy: 0.4848
Epoch 7/1000
99/99 [==============================] - 0s 91us/step - loss: 0.2473 - binary_accuracy: 0.5051
Epoch 8/1000
99/99 [==============================] - 0s 91us/step - loss: 0.2465 - binary_accuracy: 0.6869
Epoch 9/1000
99/99 [==============================] - 0s 81us/step - loss: 0.2461 - binary_accuracy: 0.6869
Epoch 10/1000
99/99 [========

99/99 [==============================] - 0s 81us/step - loss: 0.1131 - binary_accuracy: 0.8586
Epoch 76/1000
99/99 [==============================] - 0s 71us/step - loss: 0.1078 - binary_accuracy: 0.8687
Epoch 77/1000
99/99 [==============================] - 0s 71us/step - loss: 0.1080 - binary_accuracy: 0.8788
Epoch 78/1000
99/99 [==============================] - 0s 71us/step - loss: 0.1068 - binary_accuracy: 0.8889
Epoch 79/1000
99/99 [==============================] - 0s 81us/step - loss: 0.1076 - binary_accuracy: 0.8788
Epoch 80/1000
99/99 [==============================] - 0s 71us/step - loss: 0.1162 - binary_accuracy: 0.8485
Epoch 81/1000
99/99 [==============================] - 0s 81us/step - loss: 0.1126 - binary_accuracy: 0.8687
Epoch 82/1000
99/99 [==============================] - 0s 81us/step - loss: 0.1104 - binary_accuracy: 0.8788
Epoch 83/1000
99/99 [==============================] - 0s 81us/step - loss: 0.1085 - binary_accuracy: 0.8889
Epoch 84/1000
99/99 [============

99/99 [==============================] - 0s 71us/step - loss: 0.0875 - binary_accuracy: 0.8889
Epoch 150/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0875 - binary_accuracy: 0.8889
Epoch 151/1000
99/99 [==============================] - 0s 71us/step - loss: 0.1067 - binary_accuracy: 0.8788
Epoch 152/1000
99/99 [==============================] - 0s 91us/step - loss: 0.0872 - binary_accuracy: 0.9091
Epoch 153/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0873 - binary_accuracy: 0.8889
Epoch 154/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0880 - binary_accuracy: 0.9091
Epoch 155/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0872 - binary_accuracy: 0.8889
Epoch 156/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0870 - binary_accuracy: 0.8889
Epoch 157/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0856 - binary_accuracy: 0.8889
Epoch 158/1000
99/99 [===

99/99 [==============================] - 0s 91us/step - loss: 0.0682 - binary_accuracy: 0.9293
Epoch 224/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0677 - binary_accuracy: 0.9293
Epoch 225/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0653 - binary_accuracy: 0.9394
Epoch 226/1000
99/99 [==============================] - 0s 91us/step - loss: 0.0653 - binary_accuracy: 0.9293
Epoch 227/1000
99/99 [==============================] - 0s 91us/step - loss: 0.0655 - binary_accuracy: 0.9394
Epoch 228/1000
99/99 [==============================] - 0s 91us/step - loss: 0.0812 - binary_accuracy: 0.8889
Epoch 229/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0755 - binary_accuracy: 0.9091
Epoch 230/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0713 - binary_accuracy: 0.9192
Epoch 231/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0663 - binary_accuracy: 0.9293
Epoch 232/1000
99/99 [===

99/99 [==============================] - 0s 81us/step - loss: 0.0589 - binary_accuracy: 0.9293
Epoch 298/1000
99/99 [==============================] - 0s 91us/step - loss: 0.0589 - binary_accuracy: 0.9293
Epoch 299/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0576 - binary_accuracy: 0.9394
Epoch 300/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0584 - binary_accuracy: 0.9293
Epoch 301/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0579 - binary_accuracy: 0.9394
Epoch 302/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0578 - binary_accuracy: 0.9293
Epoch 303/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0581 - binary_accuracy: 0.9394
Epoch 304/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0578 - binary_accuracy: 0.9293
Epoch 305/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0651 - binary_accuracy: 0.9293
Epoch 306/1000
99/99 [===

...

99/99 [==============================] - 0s 81us/step - loss: 0.0493 - binary_accuracy: 0.9495
Epoch 668/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0493 - binary_accuracy: 0.9394
Epoch 669/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0486 - binary_accuracy: 0.9495
Epoch 670/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0490 - binary_accuracy: 0.9394
Epoch 671/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0494 - binary_accuracy: 0.9394
Epoch 672/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0484 - binary_accuracy: 0.9394
Epoch 673/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0495 - binary_accuracy: 0.9394
Epoch 674/1000
99/99 [==============================] - 0s 91us/step - loss: 0.0558 - binary_accuracy: 0.9394
Epoch 675/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0496 - binary_accuracy: 0.9394
Epoch 676/1000
99/99 [===

99/99 [==============================] - 0s 81us/step - loss: 0.0456 - binary_accuracy: 0.9495
Epoch 742/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0450 - binary_accuracy: 0.9495
Epoch 743/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0449 - binary_accuracy: 0.9495
Epoch 744/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0453 - binary_accuracy: 0.9495
Epoch 745/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0466 - binary_accuracy: 0.9495
Epoch 746/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0451 - binary_accuracy: 0.9495
Epoch 747/1000
99/99 [==============================] - 0s 61us/step - loss: 0.0455 - binary_accuracy: 0.9495
Epoch 748/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0451 - binary_accuracy: 0.9495
Epoch 749/1000
99/99 [==============================] - 0s 61us/step - loss: 0.0450 - binary_accuracy: 0.9495
Epoch 750/1000
99/99 [===

99/99 [==============================] - 0s 81us/step - loss: 0.0415 - binary_accuracy: 0.9596
Epoch 816/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0403 - binary_accuracy: 0.9596
Epoch 817/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0401 - binary_accuracy: 0.9596
Epoch 818/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0401 - binary_accuracy: 0.9596
Epoch 819/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0400 - binary_accuracy: 0.9596
Epoch 820/1000
99/99 [==============================] - 0s 91us/step - loss: 0.0395 - binary_accuracy: 0.9596
Epoch 821/1000
99/99 [==============================] - 0s 61us/step - loss: 0.0392 - binary_accuracy: 0.9596
Epoch 822/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0394 - binary_accuracy: 0.9596
Epoch 823/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0471 - binary_accuracy: 0.9495
Epoch 824/1000
99/99 [===

99/99 [==============================] - 0s 71us/step - loss: 0.0332 - binary_accuracy: 0.9596
Epoch 890/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0329 - binary_accuracy: 0.9596
Epoch 891/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0331 - binary_accuracy: 0.9596
Epoch 892/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0327 - binary_accuracy: 0.9596
Epoch 893/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0329 - binary_accuracy: 0.9596
Epoch 894/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0326 - binary_accuracy: 0.9596
Epoch 895/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0325 - binary_accuracy: 0.9596
Epoch 896/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0325 - binary_accuracy: 0.9596
Epoch 897/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0324 - binary_accuracy: 0.9596
Epoch 898/1000
99/99 [===

99/99 [==============================] - 0s 80us/step - loss: 0.0296 - binary_accuracy: 0.9697
Epoch 963/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0302 - binary_accuracy: 0.9697
Epoch 964/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0300 - binary_accuracy: 0.9697
Epoch 965/1000
99/99 [==============================] - 0s 71us/step - loss: 0.0294 - binary_accuracy: 0.9697
Epoch 966/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0786 - binary_accuracy: 0.9192
Epoch 967/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0303 - binary_accuracy: 0.9697
Epoch 968/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0514 - binary_accuracy: 0.9394
Epoch 969/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0324 - binary_accuracy: 0.9697
Epoch 970/1000
99/99 [==============================] - 0s 81us/step - loss: 0.0301 - binary_accuracy: 0.9697
Epoch 971/1000
99/99 [===

In [94]:
y_pred = model.predict_classes(x_test)
confusion_matrix(y_test,y_pred)

array([[10,  1],
       [ 1, 13]], dtype=int64)

In [95]:
accuracy_score(y_test,y_pred)

0.92

In [96]:
prob_y = model.predict_proba(x_test)
prob_y = [p[0] for p in prob_y]
print( roc_auc_score(y_test, prob_y) )

0.922077922077922


In [97]:
df.to_csv("datos_limpios.csv",index=False)

In [98]:
pickle.dump(scaler,open("scaler.pkl","wb"))
pickle.dump(model,open("model_NN.pkl","wb"))
pickle.dump(clf,open("model_RF.pkl","wb"))

In [99]:
x_total = df.drop(['patient_age_quantile','patient_id','sars_cov_2_exam_result','patient_addmited_to_intensive_care_unit_1_yes_0_no','patient_addmited_to_regular_ward_1_yes_0_no','patient_addmited_to_semi_intensive_unit_1_yes_0_no'], axis=1)

In [100]:
x_nn = scaler.transform(x_total)
y_nn = model.predict_classes(x_nn)
y_rf = clf.predict(x_nn)

In [101]:
y_nn_str = y_nn.astype('str')

In [102]:
y_nn_str[y_nn==1] = 'positive'
y_nn_str[y_nn==0] = 'negative'

In [103]:
pickle.dump(y_nn_str,open("y_nn.pkl","wb"))
pickle.dump(y_rf,open("y_rf","wb"))

In [107]:
model == pickle.load(open("model_NN.pkl","rb"))

False

In [109]:
model.save("model_NN.h5")